# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [15]:
import numpy as np
import pandas as pd

from math import log, sqrt
from sklearn import linear_model

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('data/kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = 
['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 
 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 
 'floors_square', 'waterfront', 'view', 'condition', 
 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 
 'yr_renovated']

Find what features had non-zero weight.

In [9]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [65]:
model_all.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [13]:
testing = pd.read_csv('data/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('data/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('data/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [22]:
l1_list = np.logspace(1, 7, num=13)

for l1_penalty in l1_list:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    predictions = model.predict(validation[all_features])
    RSS = ((predictions - validation['price']) ** 2).sum()
    print ('the RSS %f is for %.2f' % (RSS, l1_penalty))

the RSS 398213327300135.062500 is for 10.00
the RSS 399041900253346.937500 is for 31.62
the RSS 429791604072559.500000 is for 100.00
the RSS 463739831045121.375000 is for 316.23
the RSS 645898733633807.000000 is for 1000.00
the RSS 1222506859427163.000000 is for 3162.28
the RSS 1222506859427163.000000 is for 10000.00
the RSS 1222506859427163.000000 is for 31622.78
the RSS 1222506859427163.000000 is for 100000.00
the RSS 1222506859427163.000000 is for 316227.77
the RSS 1222506859427163.000000 is for 1000000.00
the RSS 1222506859427163.000000 is for 3162277.66
the RSS 1222506859427163.000000 is for 10000000.00


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [26]:
model10 = linear_model.Lasso(alpha=10, normalize=True)
model10.fit(training[all_features], training['price'])
predictions = model10.predict(testing[all_features])
RSS = ((predictions - testing['price']) ** 2).sum()
print 'the RSS is ' + str(RSS)

the RSS is 9.84674025527e+13


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [29]:
np.count_nonzero(model10.coef_) + np.count_nonzero(model10.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [30]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [55]:
np.arange(100,102) + np.arange(2)

array([100, 102])

In [51]:
np.logspace(2.1, 2.4, num=20)

array([ 125.89254118,  130.5537869 ,  135.387618  ,  140.40042455,
        145.59883323,  150.98971606,  156.58019951,  162.37767392,
        168.38980324,  174.6245352 ,  181.0901118 ,  187.79508019,
        194.74830399,  201.95897502,  209.4366254 ,  217.1911402 ,
        225.2327705 ,  233.57214691,  242.22029366,  251.18864315])

In [56]:
l1_penalty_values_min = np.arange(150,158)
l1_penalty_values_max = np.arange(201,210)
nonzeros_min = [0 for i in range(len(l1_penalty_values))]
nonzeros_max = [0 for i in range(len(l1_penalty_values))]

for index, l1_penalty in enumerate(l1_penalty_values_min):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzeros_min[index] = non_zeros
    
for index, l1_penalty in enumerate(l1_penalty_values_max):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzeros_max[index] = non_zeros

In [57]:
print zip(l1_penalty_values_min, nonzeros_min)

[(150, 8), (151, 8), (152, 8), (153, 8), (154, 7), (155, 7), (156, 7), (157, 7)]


In [58]:
print zip(l1_penalty_values_max, nonzeros_max)

[(201, 7), (202, 7), (203, 7), (204, 7), (205, 7), (206, 7), (207, 6), (208, 6), (209, 6)]


In [59]:
l1_penalty_min = 153
l1_penalty_max = 207

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [61]:
l1_penalty_values = np.linspace(154, 206,20)

for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    predictions = model.predict(validation[all_features])
    RSS = ((predictions - validation['price']) ** 2).sum()
    non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    print 'penalty:' + str(l1_penalty) + ' RSS:' + str(RSS) + ' params:' + str(non_zeros)

penalty:154.0 RSS:4.39830405618e+14 params:7
penalty:156.736842105 RSS:4.40101738965e+14 params:7
penalty:159.473684211 RSS:4.40379199789e+14 params:7
penalty:162.210526316 RSS:4.40664156517e+14 params:7
penalty:164.947368421 RSS:4.40956562622e+14 params:7
penalty:167.684210526 RSS:4.4125631679e+14 params:7
penalty:170.421052632 RSS:4.41563422343e+14 params:7
penalty:173.157894737 RSS:4.41877890495e+14 params:7
penalty:175.894736842 RSS:4.42199721248e+14 params:7
penalty:178.631578947 RSS:4.42528901401e+14 params:7
penalty:181.368421053 RSS:4.42865477648e+14 params:7
penalty:184.105263158 RSS:4.43209423841e+14 params:7
penalty:186.842105263 RSS:4.43560733529e+14 params:7
penalty:189.578947368 RSS:4.43921675722e+14 params:7
penalty:192.315789474 RSS:4.44287618625e+14 params:7
penalty:195.052631579 RSS:4.44660886271e+14 params:7
penalty:197.789473684 RSS:4.45041483551e+14 params:7
penalty:200.526315789 RSS:4.45429302184e+14 params:7
penalty:203.263157895 RSS:4.45824308884e+14 params:7
pe

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [64]:
model154 = linear_model.Lasso(alpha=154, normalize=True)
model154.fit(training[all_features], training['price'])
print model154.coef_

[ -0.00000000e+00  -0.00000000e+00   1.10522651e+04   1.63241881e+02
   0.00000000e+00  -0.00000000e+00  -0.00000000e+00   0.00000000e+00
   0.00000000e+00   5.08131958e+05   4.19961061e+04   0.00000000e+00
   1.16473748e+05   0.00000000e+00   0.00000000e+00  -2.62757421e+03
   0.00000000e+00]


In [ ]:
all_features = 
['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 
 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 
 'floors_square', 'waterfront', 'view', 'condition', 
 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 
 'yr_renovated']